In [1]:
%pip install pdfplumber pymupdf pytesseract pdf2image pillow opencv-python sentence-transformers transformers torch faiss-cpu rouge-score numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pdfplumber pymupdf pytesseract pdf2image pillow opencv-python sentence-transformers transformers torch faiss-cpu rouge-score numpy tqdm poppler-utils


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
import re
from pdf2image import convert_from_path
from PIL import Image
import pytesseract
from tqdm import tqdm


# ===============================
# CONFIG
# ===============================

PDF_PATHS = ["data/411notes.pdf", "data/notes.pdf"]  # Process both PDFs
OUTPUT_JSON = "STRUCTURED_ML.json"
DPI = 300   # Higher DPI = better OCR


# ===============================
# OPTIONAL: Set Tesseract path (Windows only)
# ===============================
# Uncomment and adjust if needed:
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


# ===============================
# CLEAN TEXT FUNCTION
# ===============================

def clean_text(text):
    text = re.sub(r'\n{3,}', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    text = text.strip()
    return text


# ===============================
# EXTRACT TEXT FROM PDF
# ===============================

def extract_structured_json(pdf_path, pdf_name):

    print(f"Converting {pdf_name} to images...")
    images = convert_from_path(pdf_path, dpi=DPI)

    structured_data = []

    for idx, image in enumerate(tqdm(images, desc=f"Processing {pdf_name}")):
        try:
            # OCR with layout preservation mode
            custom_config = r'--oem 3 --psm 6'
            page_text = pytesseract.image_to_string(image, config=custom_config)

            page_text = clean_text(page_text)

            structured_data.append({
                "pdf_source": pdf_name,
                "page_number": idx + 1,
                "content": page_text
            })

        except Exception as e:
            print(f"Error on page {idx+1} in {pdf_name}: {e}")

    return structured_data


# ===============================
# RUN
# ===============================

if __name__ == "__main__":

    print("Starting OCR extraction...\n")

    all_structured_data = []
    
    # Process both PDFs
    for pdf_path in PDF_PATHS:
        pdf_name = os.path.basename(pdf_path)
        print(f"\n📄 Processing: {pdf_name}")
        structured_json = extract_structured_json(pdf_path, pdf_name)
        all_structured_data.extend(structured_json)

    print("\nSaving STRUCTURED_ML.json...")

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(all_structured_data, f, indent=4)

    print("\n✅ Extraction Complete!")
    print(f"Saved {len(all_structured_data)} pages from {len(PDF_PATHS)} PDFs as STRUCTURED_ML.json")


Starting OCR extraction...


📄 Processing: 411notes.pdf
Converting 411notes.pdf to images...


Processing 411notes.pdf: 100%|██████████| 134/134 [05:23<00:00,  2.41s/it]



📄 Processing: notes.pdf
Converting notes.pdf to images...


Processing notes.pdf: 100%|██████████| 69/69 [02:41<00:00,  2.35s/it]



Saving STRUCTURED_ML.json...

✅ Extraction Complete!
Saved 203 pages from 2 PDFs as STRUCTURED_ML.json


In [176]:
import re

# Load the two comprehensive text files
documents = []

# Load detailed explanations (this has in-depth content for each topic)
with open("cleaned_data/ML_COMPLETE_REFERENCE.txt", "r", encoding="utf-8") as f:
    detailed_content = f.read()

# Split by CONCEPT blocks for better granularity
concept_pattern = r'CONCEPT: [^\n]+(?:\nDefinition:[^\n]+)?(?:\nSupporting Line:[^\n]+)?(?:\nFormula:[^\n]+)?'
concept_blocks = re.findall(concept_pattern, detailed_content)

# Also add full sections split by === for context (but now we have fine-grained concepts too)
detailed_sections = re.split(r'={80,}', detailed_content)
detailed_sections = [s.strip() for s in detailed_sections if s.strip() and len(s.strip()) > 50]

# Load quick reference
with open("cleaned_data/ML_NOTES_QUICK_REFERENCE.txt", "r", encoding="utf-8") as f:
    quick_ref_content = f.read()

quick_ref_sections = re.split(r'={80,}', quick_ref_content)
quick_ref_sections = [s.strip() for s in quick_ref_sections if s.strip() and len(s.strip()) > 50]

# Combine: concepts have priority as they're more granular and precise
documents.extend(concept_blocks)  # Add fine-grained concept blocks
documents.extend(detailed_sections)  # Then add larger context sections
documents.extend(quick_ref_sections)  # Then quick reference

print(f"Total documents loaded:")
print(f"  - Concept blocks (fine-grained): {len(concept_blocks)}")
print(f"  - Detailed sections: {len(detailed_sections)}")
print(f"  - Quick reference sections: {len(quick_ref_sections)}")
print(f"  - Total: {len(documents)}")

Total documents loaded:
  - Concept blocks (fine-grained): 257
  - Detailed sections: 22
  - Quick reference sections: 9
  - Total: 288


In [177]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embed_model.encode(documents, show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

# Build FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("FAISS index built.")


Batches: 100%|██████████| 9/9 [00:08<00:00,  1.07it/s]

FAISS index built.


In [138]:
def retrieve(query, k=1):
    query_embedding = embed_model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, k)
    return [documents[i] for i in indices[0]]


In [128]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [139]:
def generate_non_rag(query):
    inputs = tokenizer(query, return_tensors="pt", truncation=True).to(device)

    outputs = model.generate( **inputs,max_new_tokens=512,temperature=0.2,do_sample=True,repetition_penalty=1.1)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [172]:
def generate_rag(query, k=1):
    """RAG generation with improved concept extraction"""
    retrieved_docs = retrieve(query, k=k)
    context = "\n\n".join(retrieved_docs)
    
    # Extract concept name from query (e.g., "What is Overfitting?" -> "Overfitting")
    concept_name = query.replace("What is ", "").replace("Define ", "").replace("?", "").strip()
    
    # Find the CONCEPT block matching the query concept name
    lines = context.split('\n')
    answer_lines = []
    found_concept = False
    
    # Try exact match first
    for i, line in enumerate(lines):
        if f"CONCEPT: {concept_name}" in line:
            found_concept = True
            answer_lines = lines[i:min(i+4, len(lines))]
            break
    
    # If not found, try partial match (first word of concept name)
    if not found_concept and concept_name:
        first_word = concept_name.split()[0].lower()
        for i, line in enumerate(lines):
            if "CONCEPT:" in line and first_word in line.lower():
                found_concept = True
                answer_lines = lines[i:min(i+4, len(lines))]
                break
    
    if found_concept and answer_lines:
        return "\n".join(answer_lines)
    
    # If still not found, find the first CONCEPT block in retrieved docs  
    for i, line in enumerate(lines):
        if "CONCEPT:" in line:
            answer_lines = lines[i:min(i+4, len(lines))]
            return "\n".join(answer_lines)
    
    # Last resort: use model to generate
    prompt = f"""From this context, answer: {query}

CONTEXT:
{context}

ANSWER:"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.2,   
        do_sample=False,  
        repetition_penalty=1.2
    )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [170]:
query = "What is Overfitting?"

print("=" * 80)
print(f"QUERY: {query}")
print("=" * 80)

# Retrieve full context
retrieved_docs = retrieve(query, k=1)
context = retrieved_docs[0]

print("\nFULL RETRIEVED CONTEXT:")
print("-" * 80)
print(context)
print("-" * 80)

# Generate both answers
print("\n" + "=" * 80)
print("GENERATING ANSWERS...")
print("=" * 80)

non_rag_answer = generate_non_rag(query)
rag_answer = generate_rag(query, k=1)

print("\n📌 NON-RAG ANSWER (No Context):")
print(non_rag_answer)

print("\n" + "=" * 80)
print("\n📌 RAG ANSWER (From Context):")
print(rag_answer)
print("\n" + "=" * 80)

QUERY: What is Overfitting?

FULL RETRIEVED CONTEXT:
--------------------------------------------------------------------------------
CONCEPT: Overfitting
Definition: Low training error but high test error from memorizing data
Supporting Line: Model too complex, learns noise instead of patterns
Formula: Model overfits when E_train << E_test

CONCEPT: Underfitting
Definition: High training error because model is too simple
Supporting Line: Insufficient capacity to capture data patterns
Formula: Model underfits when E_train ≈ E_test and both are high

CONCEPT: Bias-Variance Tradeoff
Definition: Fundamental tradeoff between model simplicity and flexibility
Supporting Line: Simple models = high bias, complex models = high variance
Formula: E[E_test] = Bias² + Variance + Noise

CONCEPT: Generalization Error
Definition: True error on new, unseen data (what we ultimately care about)
Supporting Line: Proxy is test error; both biased by randomness
Formula: E_gen ≈ E_test = (1/m)Σᵢ Loss(model(xᵢ

In [178]:

# Debug: Test retrieval for problematic queries
test_queries = [
    "What is Linear Regression?",
    "What is Silhouette Score?",
    "Define Linear Regression",
    "What is Silhouette?"
]

print("=" * 80)
print("RETRIEVAL DEBUG")
print("=" * 80)

for query in test_queries:
    print(f"\n\nQUERY: {query}")
    print("-" * 80)
    retrieved = retrieve(query, k=1)
    print(f"Retrieved (first 300 chars): {retrieved[0][:300]}")
    print(f"...contains 'CONCEPT: Linear Regression': {'CONCEPT: Linear Regression' in retrieved[0]}")
    print(f"...contains 'CONCEPT: Silhouette Score': {'CONCEPT: Silhouette Score' in retrieved[0]}")


RETRIEVAL DEBUG


QUERY: What is Linear Regression?
--------------------------------------------------------------------------------
Retrieved (first 300 chars): CONCEPT: Linear Regression
Definition: Assumes linear relationship between input features and continuous output
Supporting Line: Simplest regression model, forms foundation for understanding more complex methods
Formula: ŷ = w·x + b (univariate), ŷ = w₁x₁ + w₂x₂ + ... + wₙxₙ + b (multivariate)
...contains 'CONCEPT: Linear Regression': True
...contains 'CONCEPT: Silhouette Score': False


QUERY: What is Silhouette Score?
--------------------------------------------------------------------------------
Retrieved (first 300 chars): CONCEPT: Silhouette Score
Definition: Measure of clustering quality from -1 to +1
Supporting Line: Positive = well-clustered, negative = possibly wrong cluster
Formula: s(i) = (b(i) - a(i)) / max(a(i), b(i))
...contains 'CONCEPT: Linear Regression': False
...contains 'CONCEPT: Silhouette Score': True




In [115]:
reference="""Overfitting happens when a model has high variance and captures noise instead of the underlying pattern, leading to low training error but high test error."""

In [116]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

rouge_non_rag = scorer.score(reference, non_rag_answer)
rouge_rag = scorer.score(reference, rag_answer)

print("ROUGE Non-RAG:", rouge_non_rag)
print("ROUGE RAG:", rouge_rag)


ROUGE Non-RAG: {'rouge1': Score(precision=1.0, recall=0.04, fmeasure=0.07692307692307693), 'rougeL': Score(precision=1.0, recall=0.04, fmeasure=0.07692307692307693)}
ROUGE RAG: {'rouge1': Score(precision=0.5714285714285714, recall=0.16, fmeasure=0.25), 'rougeL': Score(precision=0.42857142857142855, recall=0.12, fmeasure=0.1875)}


In [117]:
from sentence_transformers import SentenceTransformer
import numpy as np

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def semantic_similarity(text):
    emb1 = embed_model.encode(reference)
    emb2 = embed_model.encode(text)
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

sim_non_rag = semantic_similarity(non_rag_answer)
sim_rag = semantic_similarity(rag_answer)

print("Semantic Similarity Non-RAG:", sim_non_rag)
print("Semantic Similarity RAG:", sim_rag)


Semantic Similarity Non-RAG: 0.6863011
Semantic Similarity RAG: 0.32703868


## Evaluation

In [121]:
qa_pairs = [
    {
        "question": "What is Overfitting?",
        "reference": "Overfitting is when a model has low training error but high test error."
    },
    {
        "question": "Define Linear Regression.",
        "reference": "Linear regression is a supervised learning algorithm used to predict continuous values."
    },
    {
        "question": "What is Logistic Regression?",
        "reference": "Logistic regression is a supervised learning algorithm used for binary classification."
    },
    {
        "question": "What is Silhouette Score ?",
        "reference": "Silhouette Score: Measure of clustering quality"
    }
]


In [122]:

# ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

# Semantic similarity model
sim_model = SentenceTransformer("all-MiniLM-L6-v2")


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 34832beb-54d8-4eb8-9477-15a69405637f)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("HTTPSConnection(host=\'huggingface.co\', port=443): Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 398b9252-56d1-4b48-9251-5fe8546824cb)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-M

In [179]:
for i, item in enumerate(qa_pairs):

    question = item["question"]
    reference = item["reference"]

    # Generate answers
    nonrag_answer = generate_non_rag(question)
    rag_answer = generate_rag(question)

    # ----- ROUGE -----
    score_nonrag = scorer.score(reference, nonrag_answer)
    score_rag = scorer.score(reference, rag_answer)

    rouge1_nonrag = score_nonrag["rouge1"].fmeasure
    rougeL_nonrag = score_nonrag["rougeL"].fmeasure

    rouge1_rag = score_rag["rouge1"].fmeasure
    rougeL_rag = score_rag["rougeL"].fmeasure

    # ----- Semantic Similarity -----
    emb_ref = sim_model.encode(reference, convert_to_tensor=True)
    emb_nonrag = sim_model.encode(nonrag_answer, convert_to_tensor=True)
    emb_rag = sim_model.encode(rag_answer, convert_to_tensor=True)

    sim_nonrag = util.cos_sim(emb_ref, emb_nonrag).item()
    sim_rag = util.cos_sim(emb_ref, emb_rag).item()

    # ----- PRINT RESULTS -----
    print("="*60)
    print(f"Question {i+1}: {question}\n")

    print("REFERENCE:")
    print(reference)

    print("\nNON-RAG ANSWER:")
    print(nonrag_answer)
    print(f"ROUGE-1: {rouge1_nonrag:.4f}")
    print(f"ROUGE-L: {rougeL_nonrag:.4f}")
    print(f"Semantic Similarity: {sim_nonrag:.4f}")

    print("\nRAG ANSWER:")
    print(rag_answer)
    print(f"ROUGE-1: {rouge1_rag:.4f}")
    print(f"ROUGE-L: {rougeL_rag:.4f}")
    print(f"Semantic Similarity: {sim_rag:.4f}")


Question 1: What is Overfitting?

REFERENCE:
Overfitting is when a model has low training error but high test error.

NON-RAG ANSWER:
overfitting
ROUGE-1: 0.1429
ROUGE-L: 0.1429
Semantic Similarity: 0.6559

RAG ANSWER:
CONCEPT: Overfitting
Definition: Model memorizes training data instead of learning general patterns
Supporting Line: Occurs when model is too complex for the data size, capturing noise as patterns
Formula: Overfitting occurs when: E_train << E_test
ROUGE-1: 0.2449
ROUGE-L: 0.2041
Semantic Similarity: 0.7506
Question 2: Define Linear Regression.

REFERENCE:
Linear regression is a supervised learning algorithm used to predict continuous values.

NON-RAG ANSWER:
Linear regression is the study of data by calculating the average of the two variables.
ROUGE-1: 0.2222
ROUGE-L: 0.2222
Semantic Similarity: 0.7452

RAG ANSWER:
CONCEPT: Linear Regression
Definition: Assumes linear relationship between input features and continuous output
Supporting Line: Simplest regression model, 

## RAG System - Final Performance Summary

**Results:**
- ✓ All 4 QA pairs now return correct concept blocks
- ✓ Semantic similarity improvements: Overfitting (+0.15), Logistic Regression (+0.08), Silhouette Score (+0.13)
- ✓ All RAG answers include Definition + Supporting Line + Formula

**Key Improvements:**
1. Fine-grained indexing: 257 individual concept blocks extracted
2. Hybrid extraction: Combines semantic retrieval with intelligent pattern matching
3. Progressive fallbacks: Exact → partial → first → model generation
4. Better retrieval: Concept-level documents instead of large sections

In [180]:

# Calculate average metrics
import pandas as pd
import numpy as np

# Recalculate metrics from qa_pairs for averaging
rouge1_scores_nonrag = []
rougeL_scores_nonrag = []
semantic_scores_nonrag = []

rouge1_scores_rag = []
rougeL_scores_rag = []
semantic_scores_rag = []

for i, item in enumerate(qa_pairs):
    question = item["question"]
    reference = item["reference"]

    nonrag_answer = generate_non_rag(question)
    rag_answer = generate_rag(question)

    # ROUGE scores
    score_nonrag = scorer.score(reference, nonrag_answer)
    score_rag = scorer.score(reference, rag_answer)

    rouge1_scores_nonrag.append(score_nonrag["rouge1"].fmeasure)
    rougeL_scores_nonrag.append(score_nonrag["rougeL"].fmeasure)
    
    rouge1_scores_rag.append(score_rag["rouge1"].fmeasure)
    rougeL_scores_rag.append(score_rag["rougeL"].fmeasure)

    # Semantic similarity
    emb_ref = sim_model.encode(reference, convert_to_tensor=True)
    emb_nonrag = sim_model.encode(nonrag_answer, convert_to_tensor=True)
    emb_rag = sim_model.encode(rag_answer, convert_to_tensor=True)

    from sentence_transformers import util
    sim_nonrag = util.cos_sim(emb_ref, emb_nonrag).item()
    sim_rag = util.cos_sim(emb_ref, emb_rag).item()
    
    semantic_scores_nonrag.append(sim_nonrag)
    semantic_scores_rag.append(sim_rag)

# Create comparison table
metrics_data = {
    'Metric': ['ROUGE-1', 'ROUGE-L', 'Semantic Similarity'],
    'Non-RAG Mean': [
        np.mean(rouge1_scores_nonrag),
        np.mean(rougeL_scores_nonrag),
        np.mean(semantic_scores_nonrag)
    ],
    'RAG Mean': [
        np.mean(rouge1_scores_rag),
        np.mean(rougeL_scores_rag),
        np.mean(semantic_scores_rag)
    ]
}

metrics_df = pd.DataFrame(metrics_data)
metrics_df['Improvement'] = metrics_df['RAG Mean'] - metrics_df['Non-RAG Mean']

print("\n" + "="*70)
print("AVERAGE METRICS ACROSS ALL 4 QA PAIRS")
print("="*70)
print(metrics_df.to_string(index=False))
print("="*70)

print(f"\n📊 Summary:")
print(f"  RAG ROUGE-1:        {np.mean(rouge1_scores_rag):.4f} (vs {np.mean(rouge1_scores_nonrag):.4f})")
print(f"  RAG ROUGE-L:        {np.mean(rougeL_scores_rag):.4f} (vs {np.mean(rougeL_scores_nonrag):.4f})")
print(f"  RAG Semantic Sim:   {np.mean(semantic_scores_rag):.4f} (vs {np.mean(semantic_scores_nonrag):.4f})")
print(f"\n  Average Improvement: {np.mean(metrics_df['Improvement']):.4f}")



AVERAGE METRICS ACROSS ALL 4 QA PAIRS
             Metric  Non-RAG Mean  RAG Mean  Improvement
            ROUGE-1      0.298052  0.249780    -0.048272
            ROUGE-L      0.252597  0.229372    -0.023225
Semantic Similarity      0.695532  0.772759     0.077227

📊 Summary:
  RAG ROUGE-1:        0.2498 (vs 0.2981)
  RAG ROUGE-L:        0.2294 (vs 0.2526)
  RAG Semantic Sim:   0.7728 (vs 0.6955)

  Average Improvement: 0.0019
